# Using the statistics module in PyPSA

The `statistics` module is used to easily extract information from your networks. This is useful when inspecting your solved networks and creating first visualizations of your results.

With the `statistics` module, you can look at different metrics of your network. A list of the implemented metrics are:
    
- Capital expenditure
- Operational expenditure
- Installed capacities
- Optimal capacities
- Supply
- Withdrawal
- Curtailment
- Capacity Factor
- Revenue
- Market value
- Energy balance

Now lets look at an example.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import pypsa

First, we open an example network we want to investigate.

In [ ]:
n = pypsa.examples.scigrid_de()

Lets run an overview of all statistics by calling:

In [ ]:
n.statistics().dropna()

So far the `statistics` are not so interesting, because we have not solved the network yet. We can only see that the network already has some installed capacities for different components.

You can see that `statistics` returns a `pandas.DataFrame`. The MultiIndex of the `DataFrame` provides the name of the network component (i.e. first entry of the MultiIndex, like *Generator, Line,...*) on the first index level. The `carrier` index level provides the carrier name of the given component. For example, in `n.generators`, we have the carriers *Brown Coal, Gas* and so on.

Now lets solve the network.

In [ ]:
n.optimize(n.snapshots[:4])

Now we can look at the `statistics` of the solved network.

In [ ]:
n.statistics().round(1)

As you can see there is now much more information available. There are still no capital expenditures in the network, because we only performed an operational optimization with this example network.

If you are interested in a specific metric, e.g. curtailment, you can run

In [ ]:
curtailment = n.statistics.curtailment()
curtailment[curtailment != 0]

Note that when calling a specific metric the `statistics` module returns a `pandas.Series`.
To find the unit of the data returned by `statistics`, you can call `attrs` on the `DataFrame` or `Series`.

In [ ]:
curtailment.attrs

So the unit of curtailment is given in `MWh`. You can also customize your request.

For this you have various options:
1. You can select the component from which you want to get the metric with the attribute `comps`. Careful, `comps` has to be a list of strings.

In [ ]:
n.statistics.supply(comps=["Generator"])

2. For metrics which have a time dimension, you can choose the aggregation method or decide to not aggregate them at all. Just use the `aggregate_time` attribute to specify what you want to do.

For example calculate the mean supply/generation per time step is

In [ ]:
n.statistics.supply(comps=["Generator"], aggregate_time="mean")

Or retrieve the supply time series by not aggregating the time series. 

In [ ]:
n.statistics.supply(comps=["Generator"], aggregate_time=False).iloc[:, :4]

3. You can choose how you want to group the components of the network and how to aggregate the groups. By default the components are grouped by their carriers and summed. However, you can change this by providing different `groupby` and `aggregate_groups` attributes.

In [ ]:
n.statistics.supply(comps=["Generator"], groupby=["bus"], aggregate_groups="max")

Now you obtained the maximal supply in one time step for every bus in the network.

The keys in the provided in the `groupby` argument is primarily referring to grouper functions registered in the `n.statistics.grouper` class. You can also provide a custom function to group the components. Let's say you want to group the components by the carrier and the price zone. The carrier grouping is already implemented in the `grouper` class, but the price zone grouping is not. You can provide a custom function to group the components by the price zone.

In [ ]:
# Create random number generator
rng = np.random.default_rng()

# Use new Generator API
n.buses["price_zone"] = rng.choice(
    [1, 2, 3, 4], size=n.buses.shape[0]
)  # add random price zones


def get_price_zone(n, c, port):
    bus = f"bus{port}"
    return n.static(c)[bus].map(n.buses.price_zone).rename("price_zone")


n.statistics.supply(
    comps=["Generator"], groupby=["carrier", get_price_zone], aggregate_groups="sum"
)

Often it is better when inspecting your network to visualize the tables. Therefore, you can easily make plots to analyze your results. For example the supply of the generators.

In [ ]:
n.statistics.supply(comps=["Generator"]).droplevel(0).div(1e3).plot.bar(
    title="Generator in GWh"
)

Or you could plot the generation time series of the generators.

In [ ]:
fig, ax = plt.subplots()
n.statistics.supply(comps=["Generator"], aggregate_time=False).droplevel(0).iloc[
    :, :4
].div(1e3).T.plot.area(
    title="Generation in GW",
    ax=ax,
    legend=False,
    linewidth=0,
)
ax.legend(bbox_to_anchor=(1, 0), loc="lower left", title=None, ncol=1)

Finally, we want to look at the energy balance of the network. The energy balance is not included in the overview of the statistics module. To calculate the energy balance, you can do

In [ ]:
n.statistics.energy_balance()

Note that there is now an additional index level called bus carrier. This is because an energy balance is defined for every bus carrier. The bus carriers you have in your network you can find by looking at `n.buses.carrier.unique()`. For this network, there is only one bus carrier which is AC and corresponds to electricity. However, you can have further bus carriers for example when you have a sector coupled network. You could have heat or CO $_2$ as bus carrier. Therefore, for many `statistics` functions you have to be careful about the units of the values and it is not always given by the `attr` object of the `DataFrame` or `Series`.

Finally, we want to plot the energy balance and the energy balance time series for electrcity which has the bus carrier AC. In a sector coupled network, you could also choose other bus carriers like H2 or heat. Note that in this example "-" represents the load in the system.

In [ ]:
fig, ax = plt.subplots()
n.statistics.energy_balance().loc[:, :, "AC"].groupby(
    "carrier"
).sum().to_frame().T.plot.bar(stacked=True, ax=ax, title="Energy Balance")
ax.legend(bbox_to_anchor=(1, 0), loc="lower left", title=None, ncol=1)

In [ ]:
fig, ax = plt.subplots()
n.statistics.energy_balance(aggregate_time=False).loc[:, :, "AC"].droplevel(0).iloc[
    :, :4
].groupby("carrier").sum().where(lambda x: np.abs(x) > 1).fillna(0).T.plot.area(
    ax=ax, title="Energy Balance Timeseries"
)
ax.legend(bbox_to_anchor=(1, 0), loc="lower left", title=None, ncol=1)